<a href="https://colab.research.google.com/github/ynkim0/Stock_and_Financial_Statements/blob/main/Financial_statement2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install OpenDartReader
!pip install -U finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import OpenDartReader
import FinanceDataReader as fdr

In [5]:
stock_list = fdr.StockListing("KRX").dropna()

In [6]:
stock_list

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,59500,2,-900,-1.49,59900,60000,59400,9036472,538557972800,355202061725000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,502000,2,-13000,-2.52,506000,514000,501000,410188,207412004250,117468000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,81100,2,-400,-0.49,80100,81300,79500,1242007,100209141200,59040991801500,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,815000,2,-12000,-1.45,827000,829000,810000,37172,30319270000,58006810000000,71174000,STK
4,006400,KR7006400006,삼성SDI,KOSPI,,650000,1,5000,0.78,638000,657000,632000,308741,199444672778,44696944500000,68764530,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678,267810,KR7267810000,앙츠,KONEX,일반기업부,1150,1,20,1.77,0,0,0,0,0,2055050000,1787000,KNX
2679,215050,KR7215050006,비엔디생활건강,KONEX,일반기업부,912,0,0,0.00,0,0,0,0,0,1934959392,2121666,KNX
2680,288490,KR7288490006,나라소프트,KONEX,일반기업부,50,3,0,0.00,51,51,46,21044,983756,1907350000,38147000,KNX
2681,267060,KR7267060002,명진홀딩스,KONEX,일반기업부,181,0,0,0.00,0,0,0,0,0,1654666524,9141804,KNX


In [7]:
stock_list[['Code', 'Name']].values

array([['005930', '삼성전자'],
       ['373220', 'LG에너지솔루션'],
       ['000660', 'SK하이닉스'],
       ...,
       ['288490', '나라소프트'],
       ['267060', '명진홀딩스'],
       ['329020', '오션스톤']], dtype=object)

In [8]:
# 숫자로 모두 변환
def convert_str_to_float(value):
    if type(value) == float: # nan의 자료형은 float임
        return value
    elif value == '-': # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(',', ''))

In [9]:
def extract_info_and_save(dart, path, year, name, code, report_type, report_type_code):
    report = dart.finstate(name, year, report_type_code)
    if report is None:
        pass
    else:
        submission_date = report['rcept_no'].iloc[0][:8]
        report = report[['fs_nm', 'account_nm', 'thstrm_dt', 'thstrm_amount', 'sj_nm']]
        report.rename({"fs_nm":"개별/연결",
                       "account_nm":"계정명",
                       "sj_nm":"재무제표명",
                       "thstrm_dt": "당기일자",
                       "thstrm_amount":"금액"}, axis = 1, inplace = True)
        report['금액'] = report['금액'].apply(convert_str_to_float)
        
        if name not in os.listdir(path):
            os.mkdir(path + "/" + name)
            
        report.to_csv(path + "/{}/{}_{}년_{}.csv".format(name, submission_date, year, report_type),
                      index = False, encoding = "euc-kr")

In [ ]:
import time
path = '/content/statement'
my_api = '9a360d3e3365eba4c46575c83fe03b7947088252'
dart = OpenDartReader(my_api)
for code, name in stock_list[['Code', 'Name']].values: 
  print(name)
  for year in range(2019, 2022):
        for report_type, report_type_code in zip(["1분기보고서", "반기보고서", "3분기보고서", "사업보고서"],
                                                  ["11013", "11012", "11014", "11011"]):
            print(name, year, report_type)
            while True:
                try:
                    extract_info_and_save(dart, path, year, name, code, report_type, report_type_code)
                    break
                    
                except:
                  pass

삼성전자
삼성전자 2019 1분기보고서
삼성전자 2019 반기보고서
삼성전자 2019 3분기보고서
삼성전자 2019 사업보고서
삼성전자 2020 1분기보고서
삼성전자 2020 반기보고서
삼성전자 2020 3분기보고서
삼성전자 2020 사업보고서
삼성전자 2021 1분기보고서
삼성전자 2021 반기보고서
삼성전자 2021 3분기보고서
삼성전자 2021 사업보고서


https://gils-lab.tistory.com/44